In [1]:
from math import fabs
import numpy as np
from sklearn.datasets import make_regression

In [2]:
SAMPLES = 100

In [3]:
np.random.seed(0)

In [4]:
(X, y, coef) = make_regression(n_samples = SAMPLES, 
                               n_features = 6, 
                               n_informative = 6, 
                               effective_rank = 2,
                               n_targets = 1, 
                               coef = True,
                               bias = 3,
                               tail_strength = 0)

In [5]:
coef

array([47.93845494, 60.57119573, 63.74622774, 72.78881584, 81.19385617,
       11.56618719])

$\displaystyle \min_{\beta}\frac{1}{n}||y-X\beta||^2_{2}+\lambda||\beta||_1 + \mu||\beta||_2^2$

$L=\frac{1}{n}\displaystyle\sum_{i=1}^{n}\left(y_i - \left(\beta_0 + \displaystyle\sum_{j=1}^{6} \beta_j x_{ij}\right)\right)^2 + \lambda\displaystyle\sum_{j=0}^{6}|\beta_j| + \mu\displaystyle\sum_{j=0}^{6} \beta_j^2$

$\frac{\partial L}{\partial \beta_0} = -\frac{2}{n}\displaystyle\sum_{i=1}^{n}\left(y_i - \beta_0 -\displaystyle\sum_{j=1}^{6} \beta_j x_{ij}\right) + \lambda\frac{\beta_0}{|\beta_0|} + 2\mu\beta_0$

And for $\beta_{k\neq 0}$:

$\frac{\partial L}{\partial \beta_{k}} = -\frac{2}{n}\displaystyle\sum_{i=1}^{n}x_{ik}\left(y_i - \beta_0 -\displaystyle\sum_{j=1}^{6} \beta_j x_{ij}\right) + \lambda\frac{\beta_k}{|\beta_k|} + 2\mu\beta_k$

In [6]:
SAMPLES = 100
K = 6

In [7]:
np.random.seed(0)

In [8]:
(X, y, coef) = make_regression(n_samples = SAMPLES, 
                               n_features = 6, 
                               n_informative = 6, 
                               effective_rank = 2,
                               n_targets = 1, 
                               coef = True,
                               bias = 3,
                               tail_strength = 0)

In [9]:
X = np.hstack((np.ones((SAMPLES, 1)), X))

In [10]:
# Auxiliary function for backtracking line search
def line_search(current_params, gradient, beta):
    current_params = np.array(current_params)
    
    def _f(params):
        b = np.power(params, 4) - 16 * np.power(params, 2) + 5 * np.array(params)
        yp = 0.5 * np.dot(xp, b)
        return np.mean(np.power(y - yp, 2), axis = 0)
        
    t = 1.0
    while _f(current_params - t * gradient) > _f(current_params) - \
                                              t/2.0 * norm(gradient, ord = 2) ** 2:
        t = t * beta
        
    return t

In [11]:
# Auxiliary function to calculate gradient
# l = lambda
# m = mu
def calculate_gradient(x, y, l, m, current_params):
    db = np.zeros(7)    
    
    # Common term
    common = (y - 
              current_params[0] - 
              np.sum(np.multiply(np.tile(current_params[1:], (SAMPLES, 1)), x[:, 1:]), axis=1))
    
    # Function for the regularisation factor
    def regularisation(param, l, m):
        return l * param / fabs(param) + 2 * m * param
    
    # db_0
    db[0] = - 2 / float(SAMPLES) * np.sum(common) + regularisation(current_params[0], l, m)

    # db_k, k != 0
    for k in range(1, 7):
        db[k] = -2 / float(SAMPLES) * np.sum(np.multiply(x[:, k], common)) + regularisation(current_params[k], l, m)
    return db

In [12]:
calculate_gradient(X, y, 1, 0.01, [1, 2, 3, 4, 5, 6, 7])

array([-2.90698589,  0.86882003,  0.87470091,  1.01822005,  1.388318  ,
        0.79615466,  0.80103953])